In [1]:
from convokit import Corpus, download
from convokit import HyperConvo
import numpy as np
import pickle

In [4]:
corpus = Corpus(filename='../longreddit_construction/long-reddit-corpus')
# corpus = Corpus(filename="/sauna/reddit_201810_raw/reddit-corpus-small/reddit-corpus-small")

## 1. First, generate the 3-way tensor (hypergraph_features x (time within comment thread) x threads in subreddit)

Let's get a sense of how we should normalize the time within threads.

In [5]:
corpus.print_summary_stats()

Number of Users: 344223
Number of Utterances: 1085877
Number of Conversations: 29487


In [6]:
next(corpus.iter_conversations()).id

'dnxhx8v'

In [10]:
subreddits = [convo.meta['original_convo_meta']['subreddit'] for convo in corpus.iter_conversations()]

In [11]:
os.chdir('convokit/tensors/data')

In [12]:
with open('subreddits.p', 'wb') as f:
    pickle.dump(subreddits, f)

In [13]:
convo_ids = [convo.id for convo in corpus.iter_conversations()]

In [14]:
with open('thread_ids.p', 'wb') as f:
    pickle.dump(convo_ids, f)

In [15]:
hc_transformers = [HyperConvo(prefix_len=i, feat_name="hyperconvo-{}".format(i)) for i in range(2, 11)]

In [16]:
len(hc_transformers)

9

In [17]:
for idx, hc in enumerate(list(reversed(hc_transformers))):
    print(10-idx)
    hc.transform(corpus)

10


/Users/calebchiam/Documents/GitHub/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:11: RuntimeWarning: invalid value encountered in long_scalars
  "norm.max": lambda l: np.max(l) / np.sum(l),
/Users/calebchiam/Documents/GitHub/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:14: RuntimeWarning: invalid value encountered in long_scalars
  "norm.2nd-largest": lambda l: np.partition(l, -2)[-2] / np.sum(l) if len(l) > 1 else np.nan,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2614: RuntimeWarning: invalid v

9
8
7
6
5
4
3
2


Now we construct the tensor:

In [18]:
tensor = np.zeros((9, 10000, 164))

In [19]:
tensor.shape

(9, 10000, 164)

In [20]:
for convo_idx, convo in enumerate(corpus.iter_conversations()):
    for hyperconvo_idx in range(2, 10+1):
        tensor[hyperconvo_idx-2][convo_idx] = list(convo.meta['hyperconvo-{}'.format(hyperconvo_idx)].values())

In [21]:
# impute NaN = -1
tensor[np.isnan(tensor)] = -1

Save the tensor for future use:

In [ ]:
os.listdir('/home/caleb/')

In [23]:
os.listdir()

['observed.npy',
 'subreddits.p',
 'neuron_factor.npy',
 'time_factor.npy',
 'thread_ids.p',
 'latent.npy',
 'trial_factor.npy']

In [24]:
with open('hypergraph_tensor.p', 'wb') as f:
    pickle.dump(tensor, f)

In [31]:
hg_features = list(next(corpus.iter_conversations()).meta['hyperconvo-10'])

In [33]:
with open('hg_features.p', 'wb') as f:
    pickle.dump(hg_features, f)